# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: Попова Мария Владимировна

Группа: MADE-DS-11

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

In [1]:
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.geometry.polygon import Polygon
import folium
from openrouteservice import client
from shapely import geometry
from shapely.geometry import shape, Polygon, mapping, MultiPolygon, LineString, Point
from shapely.ops import cascaded_union, transform
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap
import seaborn as sns
from shapely.geometry import box
import geopandas as gpd
import earthpy as et
from random import uniform
from folium.plugins import TimestampedGeoJson
import overpy
import requests
import json
import rtree.index
from math import radians, cos, sin, asin, sqrt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
afg = gpd.read_file('AFG_adm0.shp')

In [4]:
afg_geo = gpd.GeoDataFrame(afg, geometry='geometry')

In [5]:
f = afg.geometry # Полигон Афганистана

Алгоритм следующий: мы берем 2 точки с одинаковой долготой (и рандомной широтой), 
сравниваем расстояние до Афганистана, оставляем точку,
которая ближе к Афганистану. У второй точки оставляем ту же широту (и берем рандомную долготу), 
затем опять сравниваем расстояние, оставляем точку, что ближе к Афганистану. 
Повторяем цикл пока не попадем на территорию страны.

In [46]:
# Начальные точки, рандомная широта и долгота для первой точки, рандомная широта для второй точки.
x1 = Point (uniform(-180,180), uniform(-90, 90))
x2 = Point (x1.coords[0][0], uniform(-90, 90))
# Массив точек, из пары точек будем добавлять ту, которая имеет меньшую дистанцию до Афг.
points = []
# Пока не попали в полигон (Афганистан)
while x1.within(afg.geometry[0]) != True and x2.within(afg.geometry[0]) != True:
# Считаем расстояние до полигона
    dist1 = x1.distance(afg.geometry[0])
    dist2 = x2.distance(afg.geometry[0])
# Сравниваем, записываем более близкую точку. У более дальной точки оставляем ту же широту (что и у ближней точки)
    if dist1 < dist2:
        points.append(Point(x1.coords[0][1],x1.coords[0][0]))
        x2 = Point (uniform(-180,180), x1.coords[0][1])
    else:
        points.append(Point(x2.coords[0][1],x2.coords[0][0]))
        x1 = Point (uniform(-180,180), x2.coords[0][1])
# Считаем расстояние до полигона
    dist1 = x1.distance(afg.geometry[0])
    dist2 = x2.distance(afg.geometry[0])
# Сравниваем, записываем более близкую точку. У более дальной точки оставляем ту же долготу (что и у ближней точки)
    if dist1 < dist2:
        points.append(Point(x1.coords[0][1],x1.coords[0][0]))
        x2 = Point (x1.coords[0][0], uniform(-90, 90))
    else:
        points.append(Point(x2.coords[0][1],x2.coords[0][0]))
        x1 = Point (x2.coords[0][0], uniform(-90, 90))
# Запишем итоговую точку
if x1.within(afg.geometry[0]) == True:
    final_point = Point(x1.coords[0][1],x1.coords[0][0])
if x2.within(afg.geometry[0]) == True:
    final_point = Point(x2.coords[0][1],x2.coords[0][0])
points.append(final_point)

In [47]:
# Подготовимся к визуализации
long = []
latd = []
for point in points:
    latd.append(point.y)
    long.append(point.x)
gdf = gpd.GeoDataFrame()
gdf['long'] = long
gdf['latd'] = latd

In [48]:
# Добавим время для визуализации пошаговости алгоритма
gdf['date'] = pd.date_range(start='1/1/2020', periods=len(gdf), freq='D').astype(str)

In [49]:
gdf.head()

,long,latd,date
0,-12.317544,-104.607585,2020-01-01
1,-12.317544,-90.878213,2020-01-02
2,-5.059379,-90.878213,2020-01-03
3,-5.059379,-9.368651,2020-01-04
4,-5.059379,-9.368651,2020-01-05


In [50]:
features = []
for _, row in gdf.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type':'Point',
            'coordinates': [row['latd'],row['long']],
        },
        'properties': {
            'time': row["date"],
            'icon': 'circle',
        }
    }     
    features.append(feature)
myMap = folium.Map(location=[20, 0], tiles="OpenStreetMap", zoom_start=2)
folium.Circle(final_point.coords[0],radius= 100, color = 'crimson').add_to(myMap)
folium.GeoJson(afg.geometry[0]).add_to(myMap)
TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='P1D'
        , max_speed=10^6
    ).add_to(myMap)
myMap

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

# К сожалению, в этой задаче у меня не получилось в одном запросе в overpass собрать всю необходимую информацию.
Я пробовала примерно следующие запросы, но без результата


In [ ]:
[out:csv(::type, name,
         ::count, ::"count:nodes", ::"count:ways", ::"count:relations")];
area["admin_level"="2"][boundary=administrative];
foreach->.regio(
  .regio out;
  ( node(area.regio)[building = residential];
    way(area.regio)[building = residential];
    rel(area.regio)[building = residential];);
  out count; 
);


Я решила выполнить оставшееся задание хотя бы для первого пункта, поскольку Австралия, как я понимаю, и есть та искомая страна

In [63]:
# Информация по банкоматам в Австралии
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
( area["ISO3166-1"="AU"];) ->.a;
        node["amenity"="atm"];
        way["amenity"="atm"];
        relation["amenity"="atm"]
          (area.a);
         (._;>;);
out geom;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [64]:
# Создадим список точек с координатами
atm_coordinates = []
for element in data['elements']:
    if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
        atm_coordinates.append((lat, lon))

In [53]:
# Создадим Р-дерево, в цикле зададим туда координаты всех банкоматов
idx = rtree.index.Rtree()
for ind, pt in enumerate(atm_coordinates):
    idx.insert(ind, (pt[0], pt[1], pt[0], pt[1]))

In [54]:
# Найдем 5 ближайщих банкоматов к нашей точке в Афганистане
res = list(idx.nearest((final_point.x,final_point.y,final_point.x,final_point.y),5))

In [59]:
for element in data['elements']:
    if element['type'] == 'node':
        print (node)

In [23]:
# Воспользуемся функцией из первой домашней работы и посчитаем сумму расстояний
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon/2) ** 2
    c = 2 * asin(sqrt(a)) 
    r = 6371
    return c * r
distance = 0   
for i in res:
    distance += haversine(final_point.y, final_point.x, atm_coordinates[i][1], atm_coordinates[i][0])
print (distance)

42224.49789023226


## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [519]:
api_key = 'key has been removed'
#Создадим карту с центром в центральном парке
clnt = client.Client(key=api_key) 
map1 = folium.Map(location=([40.76508,-73.9731]), zoom_start=12)

In [520]:
# Отметим начальную и конечную точки
central_park = [-73.9731,40.76508]
bridge_view =[-73.98959, 40.70319]

folium.map.Marker(list(reversed(central_park)), popup = 'Central Park').add_to(map1)
folium.map.Marker(list(reversed(bridge_view)), popup = 'Bridge view').add_to(map1) 

map1

In [521]:
# Получим координаты школ в окрестностях Манхеттена
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];

(
  node[amenity=school](40.6849, -74.029, 40.7717, -73.9507);
  way(area)[amenity=school](40.6849, -74.029, 40.7717, -73.9507);
  rel(area)[amenity=school](40.6849, -74.029, 40.7717, -73.9507);
);
out geom;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()
school_coordinates = []
for element in data['elements']:
    if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
        school_coordinates.append((lon, lat))

In [524]:
# Функция для создания полигонов вокруг школ
def CreateBufferPolygon(point_in, resolution=2, radius=20):    
    sr_wgs = pyproj.Proj(init='epsg:4326') # WGS84
    sr_utm = pyproj.Proj(init='epsg:32632') # UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in)
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution)
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point))
    return poly_wgs


# Функция возвращающая маршруты в обход полигонов
def CreateRoute(avoided_point_list, n=0):
    route_request = {'coordinates': coordinates, 
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                    'instructions': False,
                     'options': {'avoid_polygons': geometry.mapping(MultiPolygon(avoided_point_list))}} 
    route_directions = clnt.directions(**route_request)
    return route_directions


# Создаем так же небольшой буффер для наших дорог (будем искать пересечения полигогов)
def CreateBuffer(route_directions): 
    line_tup = []
    for line in route_directions['features'][0]['geometry']['coordinates']:
        tup_format = tuple(line)
        line_tup.append(tup_format)  
    new_linestring = LineString(line_tup)
    dilated_route = new_linestring.buffer(0.001)   
    return dilated_route

In [526]:
# Создадим полигоны вокруг школ
schools_avoidence = [] 
school_geometry = []
# Визуализируем каждую школу, создадим полигон для нее
for data in school_coordinates:
    folium.Marker(list(reversed(data)), popup = "school").add_to(map1)
    school_avoidence = CreateBufferPolygon(data,
                                           resolution=2, 
                                           radius=150)
    schools_avoidence.append(school_avoidence)
    poly = Polygon(school_avoidence)  
    school_geometry.append(poly)
# Добавим все в мультиполигон
union_poly = mapping(cascaded_union(school_geometry)) 
# Визуализируем мультиполигон (буфферные места вокруг школ, где могут появляться дети)
folium.features.GeoJson(data=union_poly,
                        name='School areas',
                        style_function=style_function('#ff00ff')).add_to(map1)
map1

In [527]:
# Построим сначала маршрут без объезда школ
avoided_point_list = [] # Сюда будем записывать школы, которые попались нам на пути
route_directions = CreateRoute(avoided_point_list) 
folium.features.GeoJson(data=route_directions,
                        name='Regular Route',
                        style_function=style_function('#ff5050'),
                        overlay=True).add_to(map1)
print('Generated regular route.')

#  Строим маршрут с учетом объездов школ
dilated_route = CreateBuffer(route_directions)
try:
    for site_poly in schools_avoidence:
        poly = Polygon(site_poly)
        # Если нашли пересечение полигона школы и нашего маршрута, строим обходной путь
        if poly.intersects(dilated_route):
            avoided_point_list.append(poly)
            route_directions = CreateRoute(avoided_point_list, 1)
            dilated_route = CreateBuffer(route_directions)
# Визуализируем макршрут с объездом школ
    folium.features.GeoJson(data=route_directions,
                            name='Alternative Route',
                            style_function=style_function('#006600'),
                            overlay=True).add_to(map1)
    print('Generated alternative route, which avoids affected areas.')
map1.add_child(folium.map.LayerControl())
map1

Generated regular route.
Generated alternative route, which avoids affected areas.
